## Query Feedback

In [1]:
from typing import List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

### 후속 질문 생성

In [11]:
class GenerateQuestions(BaseModel):
    questions: List[str] = Field(description="The answer to the user's question")
    
def generate_feedback(query: str, model_name: str, max_feedbacks: int = 3) -> List[str]:
    feedback_llm = ChatGoogleGenerativeAI(model=model_name)
    
    feedback_prompt = PromptTemplate.from_template(
        """
        Given the following query from the user, ask some follow up questions to clarify the research direction. 
        Return a maximum of ${max_feedbacks} questions, but feel free to return less if the original query is clear.
        ask the follow up questions in korean
        <query>${query}</query>`
        # Answer:\n\n
        {format_instructions}
        """
    )
    json_parser = JsonOutputParser(pydantic_object=GenerateQuestions)
    
    feedback_prompt = feedback_prompt.partial(format_instructions=json_parser.get_format_instructions())
    
    feedback_chain = feedback_prompt | feedback_llm | json_parser
    
    follow_up_questions = feedback_chain.invoke({"query": query, "max_feedbacks": max_feedbacks})
    
    try:
        if follow_up_questions is None:
            print("후속 질문이 생성되지 않았습니다.")
            return []
        questions = follow_up_questions["questions"]
        print(f"주제 {query}에 대한 후속 질문을 {max_feedbacks}개 생성했습니다.")
        print(f"생성된 후속 질문: {questions}")
        return questions
    except Exception as e:
        print(f"오류: JSON 응답 처리 중 문제 발생: {e}")
        print(f"원시 응답: {follow_up_questions}")
        print(f"오류: 쿼리 '{query}'에 대한 JSON 응답 처리 실패")
        return []

In [12]:
follow_up_questions = generate_feedback(query="RAG의 최근 동향을 알려줘.", model_name="gemini-1.5-flash-8b", max_feedbacks=3)
follow_up_questions

주제 RAG의 최근 동향을 알려줘.에 대한 후속 질문을 3개 생성했습니다.
생성된 후속 질문: ['RAG의 어떤 측면에 대해서 알고 싶으신가요? (예: 기술적 동향, 시장 점유율, 투자 동향 등)', '특정 기간(예: 지난 6개월, 최근 1년)에 대한 정보가 필요하신가요?', 'RAG에 대한 정보를 어떻게 활용하실 예정인가요?']


['RAG의 어떤 측면에 대해서 알고 싶으신가요? (예: 기술적 동향, 시장 점유율, 투자 동향 등)',
 '특정 기간(예: 지난 6개월, 최근 1년)에 대한 정보가 필요하신가요?',
 'RAG에 대한 정보를 어떻게 활용하실 예정인가요?']